In [1]:
import numpy
import time
import pandas

import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
from src.data import LorenzSystem
from src.visualisation import plot_progress, plot_param_process, plot_error, plot_param_distribution

from src.pinn import PhysicsInformedNeuralNet
from src.kalmanfilter import KalmanFilter

Using backend: pytorch



In [2]:
base_dir = '../figures/comparison'

In [3]:
def reorder_dict(d):
    return {(i, k): {j: d[i][j][k] for j in d[i].keys()}
            for i in d.keys()
            for j in d[i].keys()
            for k in d[i][j].keys()}

In [4]:
def get_kalman_filter(lorenz_true, lorenz_noise, save_dir, epochs=50000):
    t0 = time.time()
    kf = KalmanFilter(differential_equation=lorenz_noise)
    kf.train(epochs=epochs)
    analysis, forecast = kf.predict()
    computation_time = time.time() - t0
    state_cost = sum(sum(abs(lorenz_true.data.values - forecast))/len(lorenz_true.data.values))/3
    plot_progress(time=numpy.array([[t] for t in kf.input_data.index]),
                  observation=kf.differential_equation.data,
                  true=lorenz_true.data,
                  prediction=forecast,
                  analysis=analysis,
                  save_path=f"{save_dir}/kalman-state-estimation")
    param_estimate = kf.df_progress[['rho', 'sigma', 'beta']].iloc[-1]
    param_estimate.to_csv(f"{save_dir}/kalman-param.csv")
    param_cost = sum(abs(param_estimate.values - [*kf.differential_equation.param.values()]))/len(param_estimate.values)
    plot_param_process(true_params=kf.differential_equation.param,
                       df_progress=kf.df_progress,
                       save_path=f"{save_dir}/kalman-parameter-estimation")
    return {'state cost': state_cost, 'param cost': param_cost, 'time (in s)': computation_time}

In [5]:
def get_pinn(lorenz_true, lorenz_noise, save_dir, epochs=50000):
    t0 = time.time()
    pinn = PhysicsInformedNeuralNet(differential_equation=lorenz_noise)
    pinn.train(epochs=epochs)
    pinn.predict()
    computation_time = time.time() - t0
    state_cost = sum(sum(abs(lorenz_true.data.values - pinn.predict(pinn.input_data['t'])))/len(lorenz_true.data.values))/3
    plot_progress(time=pinn.input_data['t'],
                  observation=pinn.differential_equation.data,
                  true=lorenz_true.data,
                  prediction=pinn.predict(pinn.input_data['t']),
                  save_path=f"{save_dir}/pinn-state-estimation")
    param_estimate = pinn.get_param_history().iloc[-1]
    param_estimate.to_csv(f"{save_dir}/pinn-param.csv")
    param_cost = sum(abs(param_estimate.values - [*pinn.differential_equation.param.values()]))/len(param_estimate.values)
    plot_param_process(true_params=pinn.differential_equation.param,
                       df_progress=pinn.get_param_history(),
                       save_path=f"{save_dir}/pinn-parameter-estimation")
    return {'state cost': state_cost, 'param cost': param_cost, 'time (in s)': computation_time}

In [6]:
end_time = 3
kalman_results = {}
pinn_results = {}
for number_of_data in [100, 250, 500]:
    kalman_results[number_of_data] = {}
    pinn_results[number_of_data] = {}
    for noise in [0.0, 0.1, 0.5, 1.0]:
        save_dir = f"{base_dir}/lorenz_{number_of_data}_{str(noise).replace('.', '')}"  # add parameter to save_dir
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        lorenz_true = LorenzSystem(end_time=end_time,
                                   number_of_data=number_of_data)
        lorenz_true.set_data()
        lorenz_noise = LorenzSystem(end_time=end_time,
                                    number_of_data=number_of_data,
                                    noise_variance=noise)
        lorenz_noise.set_data()
        kalman_results[number_of_data][noise] = get_kalman_filter(lorenz_true, lorenz_noise, save_dir)
        pinn_results[number_of_data][noise] = get_pinn(lorenz_true, lorenz_noise, save_dir)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:13<00:00, 3762.27it/s]


best parameter: {'rho': 8.399287287942316, 'sigma': 16.014710176521834, 'beta': 1.9472047443246976}
save figure to ../figures/comparison/lorenz_100_00/kalman-state-estimation
save figure to ../figures/comparison/lorenz_100_00/kalman-parameter-estimation
Compiling model...
'compile' took 0.000118 s

Training model...

Step      Train loss                                                                                    Test loss                                                                                     Test metric
0         [8.95e-02, 2.17e-01, 1.06e-01, 2.50e+01, 1.00e+02, 6.25e+02, 2.99e+01, 2.89e+01, 2.25e+02]    [8.95e-02, 2.17e-01, 1.06e-01, 2.50e+01, 1.00e+02, 6.25e+02, 2.99e+01, 2.89e+01, 2.25e+02]    []  
1000      [8.53e-07, 4.71e-04, 1.63e-04, 2.13e+01, 5.74e+00, 2.56e+01, 2.93e+01, 1.07e+01, 3.64e+01]    [8.53e-07, 4.71e-04, 1.63e-04, 2.13e+01, 5.74e+00, 2.56e+01, 2.93e+01, 1.07e+01, 3.64e+01]    []  
2000      [1.05e-07, 4.62e-04, 1.52e-05, 2.14e+01, 5.73e+00, 2.55

38000     [9.71e-01, 8.52e-02, 3.95e-01, 1.28e-06, 1.33e-08, 1.46e-07, 2.64e+01, 1.86e+00, 8.13e+00]    [9.71e-01, 8.52e-02, 3.95e-01, 1.28e-06, 1.33e-08, 1.46e-07, 2.64e+01, 1.86e+00, 8.13e+00]    []  
39000     [9.80e-01, 7.73e-02, 3.95e-01, 2.26e-07, 2.30e-08, 3.73e-10, 2.64e+01, 1.78e+00, 8.14e+00]    [9.80e-01, 7.73e-02, 3.95e-01, 2.26e-07, 2.30e-08, 3.73e-10, 2.64e+01, 1.78e+00, 8.14e+00]    []  
40000     [9.88e-01, 7.31e-02, 3.97e-01, 1.31e-05, 2.19e-05, 2.11e-06, 2.64e+01, 1.70e+00, 8.13e+00]    [9.88e-01, 7.31e-02, 3.97e-01, 1.31e-05, 2.19e-05, 2.11e-06, 2.64e+01, 1.70e+00, 8.13e+00]    []  
41000     [9.96e-01, 9.69e-02, 3.92e-01, 2.66e-05, 1.35e-06, 1.01e-06, 2.63e+01, 1.62e+00, 8.13e+00]    [9.96e-01, 9.69e-02, 3.92e-01, 2.66e-05, 1.35e-06, 1.01e-06, 2.63e+01, 1.62e+00, 8.13e+00]    []  
42000     [9.86e-01, 6.72e-02, 3.95e-01, 7.35e-06, 4.38e-06, 3.93e-07, 2.63e+01, 1.57e+00, 8.12e+00]    [9.86e-01, 6.72e-02, 3.95e-01, 7.35e-06, 4.38e-06, 3.93e-07, 2.63e+01, 1.57e+00, 8.1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:12<00:00, 3986.62it/s]


best parameter: {'rho': 8.002686326270268, 'sigma': 16.018555769675043, 'beta': 2.1130563344741216}
save figure to ../figures/comparison/lorenz_100_01/kalman-state-estimation
save figure to ../figures/comparison/lorenz_100_01/kalman-parameter-estimation
Compiling model...
'compile' took 0.000114 s

Training model...

Step      Train loss                                                                                    Test loss                                                                                     Test metric
0         [8.82e-03, 1.35e-01, 7.06e-02, 2.50e+01, 1.00e+02, 6.25e+02, 2.41e+01, 2.95e+01, 2.43e+02]    [8.82e-03, 1.35e-01, 7.06e-02, 2.50e+01, 1.00e+02, 6.25e+02, 2.41e+01, 2.95e+01, 2.43e+02]    []  
1000      [2.49e-06, 1.93e-03, 2.90e-04, 2.13e+01, 5.75e+00, 2.55e+01, 2.94e+01, 1.07e+01, 3.64e+01]    [2.49e-06, 1.93e-03, 2.90e-04, 2.13e+01, 5.75e+00, 2.55e+01, 2.94e+01, 1.07e+01, 3.64e+01]    []  
2000      [3.17e-07, 3.70e-04, 2.26e-05, 2.13e+01, 5.74e+00, 2.54

...

best parameter: {'rho': 9.597723741323481, 'sigma': 16.125774841051065, 'beta': 2.142421374181902}
save figure to ../figures/comparison/lorenz_500_05/kalman-state-estimation
save figure to ../figures/comparison/lorenz_500_05/kalman-parameter-estimation
Compiling model...
'compile' took 0.000127 s

Training model...

Step      Train loss                                                                                    Test loss                                                                                     Test metric
0         [1.68e-03, 2.04e-03, 1.68e-01, 2.50e+01, 1.00e+02, 6.25e+02, 2.52e+01, 3.30e+01, 2.13e+02]    [1.68e-03, 2.04e-03, 1.68e-01, 2.50e+01, 1.00e+02, 6.25e+02, 2.52e+01, 3.30e+01, 2.13e+02]    []  
1000      [3.93e-06, 4.98e-04, 2.34e-04, 2.14e+01, 5.74e+00, 2.59e+01, 2.93e+01, 1.08e+01, 3.67e+01]    [3.93e-06, 4.98e-04, 2.34e-04, 2.14e+01, 5.74e+00, 2.59e+01, 2.93e+01, 1.08e+01, 3.67e+01]    []  
2000      [5.19e-07, 5.78e-04, 2.45e-05, 2.14e+01, 5.73e+00, 2.58e

38000     [1.53e-02, 4.03e-02, 2.90e-02, 2.31e-03, 1.96e-05, 1.40e-03, 3.16e-01, 2.78e-01, 3.62e-01]    [1.53e-02, 4.03e-02, 2.90e-02, 2.31e-03, 1.96e-05, 1.40e-03, 3.16e-01, 2.78e-01, 3.62e-01]    []  
39000     [8.97e-03, 2.25e-02, 1.41e-02, 5.65e-04, 1.10e-05, 3.48e-04, 2.88e-01, 2.62e-01, 2.51e-01]    [8.97e-03, 2.25e-02, 1.41e-02, 5.65e-04, 1.10e-05, 3.48e-04, 2.88e-01, 2.62e-01, 2.51e-01]    []  
40000     [6.20e-03, 1.41e-02, 7.19e-03, 9.16e-05, 3.78e-05, 3.55e-05, 2.79e-01, 2.58e-01, 2.32e-01]    [6.20e-03, 1.41e-02, 7.19e-03, 9.16e-05, 3.78e-05, 3.55e-05, 2.79e-01, 2.58e-01, 2.32e-01]    []  
41000     [6.00e-03, 1.02e-02, 4.91e-03, 4.23e-07, 6.47e-05, 1.33e-05, 2.77e-01, 2.58e-01, 2.29e-01]    [6.00e-03, 1.02e-02, 4.91e-03, 4.23e-07, 6.47e-05, 1.33e-05, 2.77e-01, 2.58e-01, 2.29e-01]    []  
42000     [5.56e-03, 7.68e-03, 3.27e-03, 8.70e-06, 5.24e-05, 8.01e-06, 2.77e-01, 2.58e-01, 2.29e-01]    [5.56e-03, 7.68e-03, 3.27e-03, 8.70e-06, 5.24e-05, 8.01e-06, 2.77e-01, 2.58e-01, 2.2

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [01:49<00:00, 457.52it/s]


best parameter: {'rho': 12.494550590635928, 'sigma': 15.924598322097602, 'beta': 2.1683351790785443}
save figure to ../figures/comparison/lorenz_500_10/kalman-state-estimation
save figure to ../figures/comparison/lorenz_500_10/kalman-parameter-estimation
Compiling model...
'compile' took 0.000122 s

Training model...

Step      Train loss                                                                                    Test loss                                                                                     Test metric
0         [1.58e-02, 2.11e-01, 5.47e-02, 2.50e+01, 1.00e+02, 6.25e+02, 2.95e+01, 3.59e+01, 2.22e+02]    [1.58e-02, 2.11e-01, 5.47e-02, 2.50e+01, 1.00e+02, 6.25e+02, 2.95e+01, 3.59e+01, 2.22e+02]    []  
1000      [6.69e-07, 2.26e-03, 2.09e-04, 2.13e+01, 5.74e+00, 2.58e+01, 3.11e+01, 1.15e+01, 3.72e+01]    [6.69e-07, 2.26e-03, 2.09e-04, 2.13e+01, 5.74e+00, 2.58e+01, 3.11e+01, 1.15e+01, 3.72e+01]    []  
2000      [8.57e-08, 6.89e-04, 1.24e-05, 2.14e+01, 5.73e+00, 2.5

38000     [5.06e+00, 8.32e-01, 1.70e+00, 8.89e+00, 2.79e-01, 7.29e-01, 2.81e+01, 9.93e+00, 1.16e+01]    [5.06e+00, 8.32e-01, 1.70e+00, 8.89e+00, 2.79e-01, 7.29e-01, 2.81e+01, 9.93e+00, 1.16e+01]    []  
39000     [4.90e+00, 8.95e-01, 1.63e+00, 8.56e+00, 2.90e-01, 7.18e-01, 2.82e+01, 1.06e+01, 1.13e+01]    [4.90e+00, 8.95e-01, 1.63e+00, 8.56e+00, 2.90e-01, 7.18e-01, 2.82e+01, 1.06e+01, 1.13e+01]    []  
40000     [4.71e+00, 9.67e-01, 1.55e+00, 8.19e+00, 3.02e-01, 7.04e-01, 2.82e+01, 1.13e+01, 1.09e+01]    [4.71e+00, 9.67e-01, 1.55e+00, 8.19e+00, 3.02e-01, 7.04e-01, 2.82e+01, 1.13e+01, 1.09e+01]    []  
41000     [4.50e+00, 1.11e+00, 1.43e+00, 7.78e+00, 3.14e-01, 6.78e-01, 2.83e+01, 1.20e+01, 1.05e+01]    [4.50e+00, 1.11e+00, 1.43e+00, 7.78e+00, 3.14e-01, 6.78e-01, 2.83e+01, 1.20e+01, 1.05e+01]    []  
42000     [4.32e+00, 1.13e+00, 1.35e+00, 7.35e+00, 3.25e-01, 6.67e-01, 2.84e+01, 1.28e+01, 1.01e+01]    [4.32e+00, 1.13e+00, 1.35e+00, 7.35e+00, 3.25e-01, 6.67e-01, 2.84e+01, 1.28e+01, 1.0

In [7]:
kalman_df = pandas.DataFrame.from_dict(reorder_dict(kalman_results)).round(3)
kalman_df.to_csv(f"{base_dir}/kalman-results.csv")
print('Kalman results', kalman_df)

Kalman results            100                                    250             \
    state_cost param_cost computation_time state_cost param_cost   
0.0      0.090      0.156           14.600      0.022      0.176   
0.1      0.115      0.045           13.854      0.061      0.122   
0.5      0.310      0.229           13.851      0.257      0.017   
1.0      0.514      0.371           14.176      0.508      0.780   

                            500                              
    computation_time state_cost param_cost computation_time  
0.0           40.068      0.010      0.119          108.624  
0.1           41.483      0.054      0.248          106.689  
0.5           40.134      0.261      0.622          110.608  
1.0           41.842      0.494      1.579          110.731  


In [8]:
kalman_df

100                                    250             \
    state_cost param_cost computation_time state_cost param_cost   
0.0      0.090      0.156           14.600      0.022      0.176   
0.1      0.115      0.045           13.854      0.061      0.122   
0.5      0.310      0.229           13.851      0.257      0.017   
1.0      0.514      0.371           14.176      0.508      0.780   

                            500                              
    computation_time state_cost param_cost computation_time  
0.0           40.068      0.010      0.119          108.624  
0.1           41.483      0.054      0.248          106.689  
0.5           40.134      0.261      0.622          110.608  
1.0           41.842      0.494      1.579          110.731

In [9]:
pinn_df = pandas.DataFrame.from_dict(reorder_dict(pinn_results)).round(3)
pinn_df.to_csv(f"{base_dir}/pinn-results.csv")
print('pinn results', pinn_df)

pinn results            100                               250                         \
    state cost param cost time (in s) state cost param cost time (in s)   
0.0      2.723     16.781     107.805      0.146      0.577     169.927   
0.1      2.709     17.282     107.150      0.011      0.033     167.492   
0.5      2.694     17.782     104.346      0.025      0.030     166.168   
1.0      2.729     16.746     104.461      2.704     19.704     157.230   

           500                         
    state cost param cost time (in s)  
0.0      0.006      0.017     304.614  
0.1      0.016      0.017     302.727  
0.5      0.017      0.027     305.250  
1.0      2.572      4.247     310.074  


In [56]:
pinn_df

100                               250                         \
    state cost param cost time (in s) state cost param cost time (in s)   
0.0      2.723     16.781     107.805      0.146      0.577     169.927   
0.1      2.709     17.282     107.150      0.011      0.033     167.492   
0.5      2.694     17.782     104.346      0.025      0.030     166.168   
1.0      2.729     16.746     104.461      2.704     19.704     157.230   

           500                         
    state cost param cost time (in s)  
0.0      0.006      0.017     304.614  
0.1      0.016      0.017     302.727  
0.5      0.017      0.027     305.250  
1.0      2.572      4.247     310.074